In [ ]:
#Experimentation and testing for capturing tidal highs and lows

In [ ]:
import pandas as pd
import numpy as np
import numba

from vtools.functions.filter import cosine_lanczos

In [ ]:
import hvplot.pandas

In [ ]:
def get_smoothed_resampled(df, cutoff_period='2H', resample_period='1T', interpolate_method='pchip'):
    '''
    '''
    dfb = df.resample(resample_period).fillna(method='backfill')
    df = df.resample(resample_period).interpolate(method=interpolate_method)
    df[dfb.iloc[:, 0].isna()] = np.nan
    return cosine_lanczos(df, cutoff_period)

In [ ]:
def localmax(df):
    df.index = pd.RangeIndex(len(df.index))
    idx=df.idxmax()
    if idx == len(df)/2:
        return df.loc[idx]
    else:
        return np.NaN

def localmin(df):
    df.index = pd.RangeIndex(len(df.index))
    idx=df.idxmin()
    if idx == len(df)/2:
        return df.loc[idx]
    else:
        return np.NaN

def periods_per_window(moving_window_size, resample_period):
    return int(pd.Timedelta(moving_window_size)/pd.Timedelta(resample_period))

def tidal_highs(df, moving_window_size='7H', resample_period='1T'):
    periods= periods_per_window(moving_window_size,resample_period)
    dfmax=dfs.rolling(moving_window_size,min_periods=periods).agg(localmax)
    dfmax=dfmax.shift(periods=-(periods//2-1))
    return dfmax.dropna()

def tidal_lows(df, moving_window_size='7H', resample_period='1T'):
    periods= periods_per_window(moving_window_size,resample_period)
    dfmin=dfs.rolling(moving_window_size,min_periods=periods).agg(localmin)
    dfmin=dfmin.shift(periods=-(periods//2-1))
    return dfmin.dropna()

In [ ]:
@numba.jit(nopython=True)
def lmax(arr):
    '''Local maximum: Returns value only when centered on maximum
    '''
    idx=np.argmax(arr)
    if idx == len(arr)/2:
        return arr[idx]
    else:
        return np.NaN

@numba.jit(nopython=True)
def lmin(arr):
    '''Local minimum: Returns value only when centered on minimum
    '''
    idx=np.argmin(arr)
    if idx == len(arr)/2:
        return arr[idx]
    else:
        return np.NaN

def periods_per_window(moving_window_size, resample_period):
    return int(pd.Timedelta(moving_window_size)/pd.Timedelta(resample_period))

def tidal_highs(df, moving_window_size='7H', resample_period='1T'):
    periods= periods_per_window(moving_window_size,resample_period)
    dfmax=dfs.rolling(moving_window_size,min_periods=periods).apply(lmax,raw=True)
    dfmax=dfmax.shift(periods=-(periods//2-1))
    return dfmax.dropna()

def tidal_lows(df, moving_window_size='7H', resample_period='1T'):
    periods= periods_per_window(moving_window_size,resample_period)
    dfmin=dfs.rolling(moving_window_size,min_periods=periods).apply(lmin,raw=True)
    dfmin=dfmin.shift(periods=-(periods//2-1))
    return dfmin.dropna()


In [ ]:
df=pd.read_csv('../tests/tidal_signal_with_disturbances.csv',index_col=0,parse_dates=[0])
df.index.freq=pd.infer_freq(df.index)

dfs=get_smoothed_resampled(df)

df.hvplot(label='original')*dfs.hvplot(label='smooth')

In [ ]:
dfh,dfl=tidal_highs(dfs),tidal_lows(dfs)

In [ ]:
dfh.hvplot.scatter()*dfl.hvplot.scatter()*df.hvplot()

In [ ]:
df=pd.read_csv('../tests/tidal_signal_with_gaps.csv',index_col=0,parse_dates=[0])
df.index.freq=pd.infer_freq(df.index)

dfs=get_smoothed_resampled(df)

df.hvplot(label='original')*dfs.hvplot(label='smooth')

In [ ]:
dfh,dfl=tidal_highs(dfs),tidal_lows(dfs)

In [ ]:
dfh.hvplot.scatter()*dfl.hvplot.scatter()*dfs.hvplot()